In [ ]:
# Processing Mother's Milk Deblur BIOM Table from Qiita (05Jan21)

In [15]:
cd /Users/tanya/Dropbox/00.CU Boulder/MM Microbiome Sequences/Qiita Output/study_11908_121420-111241/BIOM/62060 deblur reference hit table/Processed files by TLA

/Users/tanya/Dropbox/00.CU Boulder/MM Microbiome Sequences/Qiita Output/study_11908_121420-111241/BIOM/62060 deblur reference hit table/Processed files by TLA


In [16]:
#convert biom file to txt to check contents
#contains OTU sequences as rows and samples as cols (including blanks/controls)
!biom convert -i reference-hit.biom -o reference-hit.txt --to-tsv

# Import deblurred sequences (fasta) and assign taxonomy - Using  Greengenes
### Assign taxonomy to deblurred sequences using Greengenes 13_5 trimmed to the V4 region

NOTE: This classifier was obtained from:
http://greengenes.secondgenome.com/downloads/database/13_5
***

In [43]:
#Import deblur sequences
!qiime tools import \
--input-path reference-hit.seqs.fa \
--type "FeatureData[Sequence]" \
--output-path mm_150bp_deblur_reference-hit_seqs.qza

Imported reference-hit.seqs.fa as DNASequencesDirectoryFormat to mm_150bp_deblur_reference-hit_seqs.qza


In [24]:
#This function requires a sequence table to assign taxonomy
#Input of deblur-seqs.qza is for all sequences, which will be applied to processed freq table below
!qiime feature-classifier classify-sklearn \
--i-reads mm_150bp_deblur_reference-hit_seqs.qza \
--i-classifier gg-13-8-99-515-806-nb-classifier.qza \
--o-classification mm_150bp_deblur_reference-hit_seqs_taxonomy_gg.qza
#Using classifer for V4 region, downloaded from "Data Resources" on Qiime2 page

Saved FeatureData[Taxonomy] to: mm_150bp_deblur_reference-hit_seqs_taxonomy_gg.gza.qza


In [33]:
# Output taxonomy .qza file as .tsv
!qiime tools export \
  --input-path mm_150bp_deblur_reference-hit_seqs_taxonomy_gg.qza \
  --output-path mm_150bp_deblur_reference-hit_seqs_taxonomy_gg

Exported mm_150bp_deblur_reference-hit_seqs_taxonomy_gg.qza as TSVTaxonomyDirectoryFormat to directory mm_150bp_deblur_reference-hit_seqs_taxonomy_gg


## Filter sOTU table to produce tables for analysis

In [25]:
!qiime tools import \
--input-path reference-hit.biom \
--input-format BIOMV210Format \
--type 'FeatureTable[Frequency]' \
 --output-path mm_150bp_deblur.qza

!qiime feature-table summarize \
--i-table mm_150bp_deblur_rg.qza \
--m-sample-metadata-file 62060_mapping_file.txt \
--o-visualization mm_150bp_deblur.qzv

Imported reference-hit.biom as BIOMV210Format to mm_150bp_deblur.qza
Saved Visualization to: mm_150bp_deblur.qzv


## Import insertion-tree generated with deblurred data (from Qiita)
***

In [26]:
#Insertion Tree from Qiita
!qiime tools import \
  --input-path insertion_tree.relabelled.tre \
  --output-path mm_150bp_deblur_insertion_tree.qza \
  --type 'Phylogeny[Rooted]'

Imported insertion_tree.relabelled.tre as NewickDirectoryFormat to mm_150bp_deblur_insertion_tree.qza


## Filter sOTU table to exclude fragments not inserted into phylogeny, and create summary of new table (referred to as the main sOTU table, or main table, from here-on-out)

In [19]:
!qiime fragment-insertion filter-features \
  --i-table mm_150bp_deblur.qza \
  --i-tree mm_150bp_deblur_insertion_tree.qza \
  --o-filtered-table mm_150bp_deblur_sepp.qza \
  --o-removed-table mm_150bp_deblur_sepp_removed.qza

!qiime feature-table summarize \
  --i-table mm_150bp_deblur_sepp_removed.qza \
  --o-visualization mm_150bp_deblur_sepp_removed.qzv

!qiime feature-table summarize \
  --i-table mm_150bp_deblur_sepp.qza \
  --o-visualization mm_150bp_deblur_sepp.qzv
# This steps takes awhile
# Makes table have same samples/dimensions

Saved FeatureTable[Frequency] to: mm_150bp_deblur_sepp.qza
Saved FeatureTable[Frequency] to: mm_150bp_deblur_sepp_removed.qza
Saved Visualization to: mm_150bp_deblur_sepp_removed.qzv
Saved Visualization to: mm_150bp_deblur_sepp.qzv


## Filter sOTU table to exclude mitochondrial reads

In [27]:
!qiime taxa filter-table \
  --i-table mm_150bp_deblur_sepp.qza \
  --i-taxonomy deblur-sequences-taxonomy-MM.qza \
  --p-exclude mitochondria \
  --o-filtered-table mm_150bp_deblur_sepp_noMit.qza

Saved FeatureTable[Frequency] to: mm_150bp_deblur_sepp_noMit.qza


## Filter sOTU table to exclude chloroplast reads, and generate summary of new feature-table
***

In [28]:
!qiime taxa filter-table \
  --i-table mm_150bp_deblur_sepp_noMit.qza \
  --i-taxonomy deblur-sequences-taxonomy-MM.qza \
  --p-exclude chloroplast \
  --o-filtered-table mm_150bp_deblur_sepp_noMit_noChl.qza

!qiime feature-table summarize \
  --i-table mm_150bp_deblur_sepp_noMit_noChl.qza \
  --o-visualization mm_150bp_deblur_sepp_noMit_noChl.qzv

Saved FeatureTable[Frequency] to: mm_150bp_deblur_sepp_noMit_noChl.qza
Saved Visualization to: mm_150bp_deblur_sepp_noMit_noChl.qzv


In [55]:
#Output freq table that contains sequences to biom file
#Remove extra header before reading into R
!qiime tools export \
 --input-path mm_150bp_deblur_sepp_noMit_noChl.qza \
 --output-path mm_150bp_deblur_sepp_noMit_noChl

# Output biom file to .tsv file
!biom convert -i mm_150bp_deblur_sepp_noMit_noChl/feature-table.biom -o mm_150bp_deblur_sepp_noMit_noChl.tsv --to-tsv

Exported mm_150bp_deblur_sepp_noMit_noChl.qza as BIOMV210DirFmt to directory mm_150bp_deblur_sepp_noMit_noChl


## Rarefy sOTU table (sampling depth of 10,000 sequences), and create summary of new table. Used for diversity measures and associated tests (e.g., permanova, biplots)
***

In [37]:
!qiime feature-table rarefy \
  --i-table mm_150bp_deblur_sepp_noMit_noChl.qza \
  --p-sampling-depth 10000 \
  --o-rarefied-table mm_150bp_deblur_sepp_noMit_noChl_rar10k.qza

!qiime feature-table summarize \
  --i-table mm_150bp_deblur_sepp_noMit_noChl_rar10k.qza \
  --o-visualization mm_150bp_deblur_sepp_noMit_noChl_rar10k.qzv

Saved FeatureTable[Frequency] to: mm_150bp_deblur_sepp_noMit_noChl_rar10k.qza
Saved Visualization to: mm_150bp_deblur_sepp_noMit_noChl_rar10k.qzv


## Create Taxonomy Tables

In [29]:
#Filter singletons from table
#Not using rareified table
!qiime feature-table filter-features \
--i-table mm_150bp_deblur_sepp_noMit_noChl.qza \
--p-min-samples 2 \
--o-filtered-table mm_150bp_deblur_sepp_noMit_noChl_rmsingletons.qza

#Summary of filtered table
!qiime feature-table summarize \
--i-table mm_150bp_deblur_sepp_noMit_noChl_rmsingletons.qza \
--m-sample-metadata-file 62060_mapping_file.txt \
--o-visualization mm_150bp_deblur_sepp_noMit_noChl_rmsingletons.qzv

Saved FeatureTable[Frequency] to: mm_150bp_deblur_sepp_noMit_noChl_rmsingletons.qza
Saved Visualization to: mm_150bp_deblur_sepp_noMit_noChl_rmsingletons.qzv


In [30]:
#Collapse taxonomy
!qiime taxa collapse \
  --i-table mm_150bp_deblur_sepp_noMit_noChl_rmsingletons.qza \
  --i-taxonomy mm_150bp_deblur_reference-hit_seqs_taxonomy_gg.qza\
  --p-level 1 \
  --o-collapsed-table mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level1.qza

!qiime taxa collapse \
  --i-table mm_150bp_deblur_sepp_noMit_noChl_rmsingletons.qza \
  --i-taxonomy mm_150bp_deblur_reference-hit_seqs_taxonomy_gg.qza \
  --p-level 2 \
  --o-collapsed-table mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level2.qza

!qiime taxa collapse \
  --i-table mm_150bp_deblur_sepp_noMit_noChl_rmsingletons.qza \
  --i-taxonomy mm_150bp_deblur_reference-hit_seqs_taxonomy_gg.qza \
  --p-level 3 \
  --o-collapsed-table mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level3.qza

!qiime taxa collapse \
  --i-table mm_150bp_deblur_sepp_noMit_noChl_rmsingletons.qza \
  --i-taxonomy mm_150bp_deblur_reference-hit_seqs_taxonomy_gg.qza \
  --p-level 4 \
  --o-collapsed-table mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level4.qza

!qiime taxa collapse \
  --i-table mm_150bp_deblur_sepp_noMit_noChl_rmsingletons.qza \
  --i-taxonomy mm_150bp_deblur_reference-hit_seqs_taxonomy_gg.qza \
  --p-level 5 \
  --o-collapsed-table mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level5.qza

!qiime taxa collapse \
  --i-table mm_150bp_deblur_sepp_noMit_noChl_rmsingletons.qza \
  --i-taxonomy mm_150bp_deblur_reference-hit_seqs_taxonomy_gg.qza \
  --p-level 6 \
  --o-collapsed-table mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level6.qza

!qiime taxa collapse \
  --i-table mm_150bp_deblur_sepp_noMit_noChl_rmsingletons.qza \
  --i-taxonomy mm_150bp_deblur_reference-hit_seqs_taxonomy_gg.qza \
  --p-level 7 \
  --o-collapsed-table mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level7.qza

Saved FeatureTable[Frequency] to: mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level1.qza
Saved FeatureTable[Frequency] to: mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level2.qza
Saved FeatureTable[Frequency] to: mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level3.qza
Saved FeatureTable[Frequency] to: mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level4.qza
Saved FeatureTable[Frequency] to: mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level5.qza
Saved FeatureTable[Frequency] to: mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level6.qza
Saved FeatureTable[Frequency] to: mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level7.qza


In [31]:
# Convert qza files to biom files
!qiime tools export \
  --input-path mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level1.qza \
  --output-path mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level1

!qiime tools export \
  --input-path mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level2.qza \
  --output-path mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level2

!qiime tools export \
  --input-path mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level3.qza \
  --output-path mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level3

!qiime tools export \
  --input-path mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level4.qza \
  --output-path mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level4

!qiime tools export \
  --input-path mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level5.qza \
  --output-path mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level5

!qiime tools export \
  --input-path mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level6.qza \
  --output-path mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level6

!qiime tools export \
  --input-path mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level7.qza \
  --output-path mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level7

Exported mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level1.qza as BIOMV210DirFmt to directory mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level1
Exported mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level2.qza as BIOMV210DirFmt to directory mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level2
Exported mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level3.qza as BIOMV210DirFmt to directory mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level3
Exported mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level4.qza as BIOMV210DirFmt to directory mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level4
Exported mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level5.qza as BIOMV210DirFmt to directory mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level5
Exported mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level6.qza as BIOMV210DirFmt to directory mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level6
Exported mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level7.qza as BIOMV210DirFmt to directory mm_

In [56]:
# Convert biom files to txt files
!biom convert -i mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level1/feature-table.biom -o mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level1.tsv --to-tsv
!biom convert -i mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level2/feature-table.biom -o mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level2.tsv --to-tsv
!biom convert -i mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level3/feature-table.biom -o mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level3.tsv --to-tsv
!biom convert -i mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level4/feature-table.biom -o mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level4.tsv --to-tsv
!biom convert -i mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level5/feature-table.biom -o mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level5.tsv --to-tsv
!biom convert -i mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level6/feature-table.biom -o mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level6.tsv --to-tsv
!biom convert -i mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level7/feature-table.biom -o mm_150bp_deblur_sepp_noMit_noChl_rmsingletons_level7.tsv --to-tsv

# Perform alpha-diversity analyses
***
## Observed OTUs, Fisher's alpha, Shannon's Index, and Faith's Phylogenetic Diversity (PD)
***
### NOTE: The output vectors from these analyses can be used as metadata input for further analyses such as in ordinations
***

In [39]:
# All data - rarefied
!qiime diversity alpha \
  --i-table mm_150bp_deblur_sepp_noMit_noChl_rar10k.qza \
  --p-metric observed_features \
  --o-alpha-diversity mm_150bp_deblur_sepp_noMit_noChl_rar10k_alpha_richness.qza

!qiime diversity alpha \
  --i-table mm_150bp_deblur_sepp_noMit_noChl_rar10k.qza \
  --p-metric shannon \
  --o-alpha-diversity mm_150bp_deblur_sepp_noMit_noChl_rar10k_alpha_shannon.qza

!qiime diversity alpha-phylogenetic \
  --i-table mm_150bp_deblur_sepp_noMit_noChl_rar10k.qza \
  --i-phylogeny mm_150bp_deblur_insertion_tree.qza \
  --p-metric faith_pd \
  --o-alpha-diversity mm_150bp_deblur_sepp_noMit_noChl_rar10k_alpha_faithspd.qza

Saved SampleData[AlphaDiversity] to: mm_150bp_deblur_sepp_noMit_noChl_rar10k_alpha_richness.qza
Saved SampleData[AlphaDiversity] to: mm_150bp_deblur_sepp_noMit_noChl_rar10k_alpha_shannon.qza
Saved SampleData[AlphaDiversity] to: mm_150bp_deblur_sepp_noMit_noChl_rar10k_alpha_faithspd.qza


In [ ]:
#Example if wanted to test group significance
#!qiime diversity alpha-group-significance \
 #--i-alpha-diversity mm_150bp_deblur_sepp_noMit_noChl_rar10k_alpha_richness.qza \
  #--m-metadata-file mm_metadata_rapid_growth_reduced_categorical.txt \
  #--o-visualization mm_150bp_deblur_sepp_noMit_noChl_rar10k_alpha_richness_group_compare.qzv

#!qiime diversity alpha-group-significance \
 # --i-alpha-diversity mm_150bp_deblur_rig_sepp_noMit_noChl_rar10k_alpha_shannon.qza \
 # --m-metadata-file mm_metadata_rapid_growth_reduced_categorical.txt \
 # --o-visualization mm_150bp_deblur_rig_sepp_noMit_noChl_rar10k_alpha_shannon_group_compare.qzv

#!qiime diversity alpha-group-significance \
  #--i-alpha-diversity mm_150bp_deblur_rig_sepp_noMit_noChl_rar10k_alpha_faithspd.qza \
  #--m-metadata-file mm_metadata_rapid_growth_reduced_categorical.txt \
  #--o-visualization mm_150bp_deblur_rig_sepp_noMit_noChl_rar10k_alpha_faithspd_group_compare.qzv

# Perform ordination analyses
***

In [46]:
!qiime diversity beta-phylogenetic \
  --i-table mm_150bp_deblur_sepp_noMit_noChl_rar10k.qza \
  --i-phylogeny mm_150bp_deblur_insertion_tree.qza \
  --p-metric unweighted_unifrac \
  --o-distance-matrix mm_150bp_deblur_sepp_noMit_noChl_rar10k_dist_unifrac.qza

!qiime diversity beta-phylogenetic \
  --i-table mm_150bp_deblur_sepp_noMit_noChl_rar10k.qza \
  --i-phylogeny mm_150bp_deblur_insertion_tree.qza \
  --p-metric weighted_unifrac \
  --o-distance-matrix mm_150bp_deblur_sepp_noMit_noChl_rar10k_dist_wunifrac.qza

Saved DistanceMatrix to: mm_150bp_deblur_sepp_noMit_noChl_rar10k_dist_unifrac.qza
Saved DistanceMatrix to: mm_150bp_deblur_sepp_noMit_noChl_rar10k_dist_wunifrac.qza


In [47]:
# May want to update metadata with Air pollution exposures
!qiime diversity pcoa \
  --i-distance-matrix mm_150bp_deblur_sepp_noMit_noChl_rar10k_dist_unifrac.qza \
  --o-pcoa mm_150bp_deblur_sepp_noMit_noChl_rar10k_pcoa_unifrac.qza

!qiime diversity pcoa \
  --i-distance-matrix mm_150bp_deblur_sepp_noMit_noChl_rar10k_dist_wunifrac.qza \
  --o-pcoa mm_150bp_deblur_sepp_noMit_noChl_rar10k_pcoa_wunifrac.qza

!qiime emperor plot \
  --i-pcoa mm_150bp_deblur_sepp_noMit_noChl_rar10k_pcoa_unifrac.qza \
  --m-metadata-file 62060_mapping_file.txt \
  --o-visualization mm_150bp_deblur_sepp_noMit_noChl_rar10k_pcoa_unifrac.qzv

!qiime emperor plot \
  --i-pcoa mm_150bp_deblur_sepp_noMit_noChl_rar10k_pcoa_wunifrac.qza \
  --m-metadata-file 62060_mapping_file.txt \
  --o-visualization mm_150bp_deblur_sepp_noMit_noChl_rar10k_pcoa_wunifrac.qzv

Saved PCoAResults to: mm_150bp_deblur_sepp_noMit_noChl_rar10k_pcoa_unifrac.qza
Saved PCoAResults to: mm_150bp_deblur_sepp_noMit_noChl_rar10k_pcoa_wunifrac.qza
Saved Visualization to: mm_150bp_deblur_sepp_noMit_noChl_rar10k_pcoa_unifrac.qzv
Saved Visualization to: mm_150bp_deblur_sepp_noMit_noChl_rar10k_pcoa_wunifrac.qzv


## Perform Robust-Aitchison Principal Components Analysis (RPCA) using DEICODE, for various sOTU tables detailed below
***

### Filter samples with fewer than 10,000 reads

In [48]:
!qiime feature-table filter-samples \
  --i-table mm_150bp_deblur_sepp_noMit_noChl.qza \
  --p-min-frequency 10000 \
  --o-filtered-table mm_150bp_deblur_sepp_noMit_noChl_min10k.qza

!qiime feature-table summarize \
  --i-table mm_150bp_deblur_sepp_noMit_noChl_min10k.qza \
  --o-visualization mm_150bp_deblur_sepp_noMit_noChl_min10k.qzv

Saved FeatureTable[Frequency] to: mm_150bp_deblur_sepp_noMit_noChl_min10k.qza
Saved Visualization to: mm_150bp_deblur_sepp_noMit_noChl_min10k.qzv


### Filter singleton and doubleton features

In [49]:
!qiime feature-table filter-features \
  --i-table mm_150bp_deblur_sepp_noMit_noChl_min10k.qza \
  --p-min-samples 3 \
  --o-filtered-table mm_150bp_deblur_sepp_noMit_noChl_min10k_noDbltns.qza

!qiime feature-table summarize \
  --i-table mm_150bp_deblur_sepp_noMit_noChl_min10k_noDbltns.qza \
  --o-visualization mm_150bp_deblur_sepp_noMit_noChl_min10k_noDbltns.qzv

Saved FeatureTable[Frequency] to: mm_150bp_deblur_sepp_noMit_noChl_min10k_noDbltns.qza
Saved Visualization to: mm_150bp_deblur_sepp_noMit_noChl_min10k_noDbltns.qzv


In [50]:
!qiime deicode rpca \
  --i-table mm_150bp_deblur_sepp_noMit_noChl_min10k_noDbltns.qza \
  --p-min-feature-count 10 \
  --p-min-sample-count 10 \
  --o-biplot mm_150bp_deblur_sepp_noMit_noChl_min10k_noDbltns_pca_rpca.qza \
  --o-distance-matrix mm_150bp_deblur_sepp_noMit_noChl_min10k_noDbltns_dist_rpca.qza

Saved PCoAResults % Properties('biplot') to: mm_150bp_deblur_sepp_noMit_noChl_min10k_noDbltns_pca_rpca.qza
Saved DistanceMatrix to: mm_150bp_deblur_sepp_noMit_noChl_min10k_noDbltns_dist_rpca.qza


In [51]:
#May want to update metadata file with AP exposues
!qiime emperor biplot \
  --i-biplot mm_150bp_deblur_sepp_noMit_noChl_min10k_noDbltns_pca_rpca.qza \
  --m-sample-metadata-file 62060_mapping_file.txt \
  --m-feature-metadata-file mm_150bp_deblur_reference-hit_seqs_taxonomy_gg.qza \
  --p-number-of-features 10 \
  --o-visualization mm_150bp_deblur_sepp_noMit_noChl_min10k_noDbltns_pca_rpca.qzv

Saved Visualization to: mm_150bp_deblur_sepp_noMit_noChl_min10k_noDbltns_pca_rpca.qzv


## Examples of Runing permdisp and permanova on rapid infant growth groups

In [ ]:
#qiime diversity beta-group-significance \
 # --i-distance-matrix mm_150bp_deblur_rig_sepp_noMit_noChl_rar10k_dist_unifrac.qza \
  #--m-metadata-file mm_metadata_rapid_growth_reduced_categorical.txt \
  #--m-metadata-column rapidGrowth \
  #--p-method permdisp \
  #--p-pairwise \
  #--p-permutations 999 \
  #--o-visualization mm_150bp_deblur_rig_sepp_noMit_noChl_rar10k_dist_unifrac_permdisp_rapidgrowth.qzv

In [ ]:
#qiime diversity beta-group-significance \
 # --i-distance-matrix mm_150bp_deblur_rig_sepp_noMit_noChl_rar10k_dist_unifrac.qza \
  #--m-metadata-file mm_metadata_rapid_growth_reduced_categorical.txt \
  #--m-metadata-column rapidGrowth \
  #--p-method permanova \
  #--p-pairwise \
  #--p-permutations 999 \
  #--o-visualization mm_150bp_deblur_rig_sepp_noMit_noChl_rar10k_dist_unifrac_permanova_rapidgrowth.qzv

## Run SONGBIRD to associate sOTUs with specific metadata groupings
### NOTE: Make sure you are running the correct Conda Environment for running SONGBIRD
### Dont forget to remove NA from Metadata or Qiime sees as categorical data (do this in xcel)

In [ ]:
# Used these files do sample triming in R (removed blanks, made metadata smaple == seq files samples)
# metadata.txt
# ddeblur-freq-table-stool-filtered-MM-rarefy10k.txt
# Need to move columns right and delete extra col in txt files from R output

Model: Exposures

In [59]:
# Import trimmed sequence file (blanks removed in R) as biom - INFANTS
!biom convert -i mm_seqs_noblank_inf.txt -o mm_seqs_noblank_inf.biom --table-type="OTU table" --to-hdf5
!biom convert -i mm_seqs_noblank_inf_1m.txt -o mm_seqs_noblank_inf_1m.biom --table-type="OTU table" --to-hdf5
!biom convert -i mm_seqs_noblank_inf_6m.txt -o mm_seqs_noblank_inf_6m.biom --table-type="OTU table" --to-hdf5
!biom convert -i mm_seqs_noblank_inf_12m.txt -o mm_seqs_noblank_inf_12m.biom --table-type="OTU table" --to-hdf5

In [60]:
# Convert imported biom seqs file to .qza - INFANTS
!qiime tools import \
--input-path mm_seqs_noblank_inf.biom \
--output-path mm_seqs_noblank_inf.qza \
--type 'FeatureTable[Frequency]' 

!qiime tools import \
--input-path mm_seqs_noblank_inf_1m.biom \
--output-path mm_seqs_noblank_inf_1m.qza \
--type 'FeatureTable[Frequency]' 

!qiime tools import \
--input-path mm_seqs_noblank_inf_6m.biom \
--output-path mm_seqs_noblank_inf_6m.qza \
--type 'FeatureTable[Frequency]' 

!qiime tools import \
--input-path mm_seqs_noblank_inf_12m.biom \
--output-path mm_seqs_noblank_inf_12m.qza \
--type 'FeatureTable[Frequency]' 

Imported mm_seqs_noblank_inf.biom as BIOMV210DirFmt to mm_seqs_noblank_inf.qza
Imported mm_seqs_noblank_inf_1m.biom as BIOMV210DirFmt to mm_seqs_noblank_inf_1m.qza
Imported mm_seqs_noblank_inf_6m.biom as BIOMV210DirFmt to mm_seqs_noblank_inf_6m.qza
Imported mm_seqs_noblank_inf_12m.biom as BIOMV210DirFmt to mm_seqs_noblank_inf_12m.qza


In [ ]:
# Generate a null model for 6-month infant gut microbiota
!qiime songbird multinomial \
--i-table mm_seqs_noblank_inf_6m.qza \
--m-metadata-file mm_meta_added_infant_6m.txt \
--p-formula "1" \
--p-epochs 10000 \
--p-differential-prior 0.5 \
--p-summary-interval 1 \
--o-differentials null-diff_6m.qza \
--o-regression-stats null-stats_6m.qza \
--o-regression-biplot null-biplot_6m.qza

In [ ]:
#Exampl
#!songbird multinomial \
 #  --input-biom mm_150bp_deblur_rig_sepp_noMit_noChl_min10k_noDbltns.biom \
 #  --metadata-file mm_metadata_rapid_growth_reduced_categorical.txt \
 #  --formula 1 \
 #  --epochs 10000 \
 #  --differential-prior 0.1 \
 #  --training-column trainingStatus \
 #  --min-feature-count 3 \
 #  --summary-interval 1 \
 #  --summary-dir 01_songbird_min10k_noDbltns_null_model

In [142]:
# Generate tree for diversity measures
!qiime alignment mafft \
  --i-sequences deblur-seqs.qza \
  --o-alignment aligned-seqs.qza

Saved FeatureData[AlignedSequence] to: aligned-seqs.qza


In [144]:
# Next, we mask (or filter) the alignment to remove positions that are highly variable. 
# These positions are generally considered to add noise to a resulting phylogenetic tree.
!qiime alignment mask \
  --i-alignment aligned-seqs.qza \
  --o-masked-alignment masked-aligned-seqs.qza

Saved FeatureData[AlignedSequence] to: masked-aligned-seqs.qza


In [145]:
# Next, we’ll apply FastTree to generate a phylogenetic tree from the masked alignment.
!qiime phylogeny fasttree \
  --i-alignment masked-aligned-seqs.qza \
  --o-tree unrooted-tree.qza

Saved Phylogeny[Unrooted] to: unrooted-tree.qza


In [146]:
# The FastTree program creates an unrooted tree, so in the final step in this section we apply midpoint 
# rooting to place the root of the tree at the midpoint of the longest tip-to-tip distance in the unrooted tree.
!qiime phylogeny midpoint-root \
  --i-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

Saved Phylogeny[Rooted] to: rooted-tree.qza


In [156]:
# Calculate Core Metric Results using 10k rarification 
# Input table needs to be a frequency table from deblur (already imported above)
# Using updated meta data for moms and infants (all timpoints)!
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table deblur-freq-table-stool-MM.qza \
  --p-sampling-depth 10000 \
  --m-metadata-file mm_meta_added_mom_infant.txt \
  --output-dir core-metrics-results10k

Saved FeatureTable[Frequency] to: core-metrics-results10k/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results10k/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results10k/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results10k/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results10k/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results10k/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results10k/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results10k/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results10k/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results10k/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results10k/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results10k/jaccard_pcoa_results.qza
Saved PCoAResults to: core

In [93]:
# Simple univariate model
# Include type row to deal with NA
!qiime songbird multinomial \
--i-table mm_seqs_noblank_inf_6m.qza \
--m-metadata-file mm_meta_added_infant_6m.txt \
--p-formula "PM25_weighted" \
--p-epochs 10000 \
--p-differential-prior 0.5 \
--p-summary-interval 1 \
--o-differentials mm_differentials-PM25weighted_6m.qza \
--o-regression-stats mm_regression-PM25weighted_6m.qza \
--o-regression-biplot mm_biplot-PM25weighted_6m.qza

# Fully adjusted model
!qiime songbird multinomial \
--i-table mm_seqs_noblank_inf_6m.qza \
--m-metadata-file mm_meta_added_infant_6m.txt \
--p-formula "PM25_weighted+prepreg_bmi_kgm2+breastfeedings_continuous+age_in_days+baby_gender_cat+baby_birthweight_kg+gestational_age_category" \
--p-epochs 10000 \
--p-differential-prior 0.5 \
--p-summary-interval 1 \
--o-differentials mm_differentials-PM25weighted_6m_preprgBMI_BFcont_age_sex_birthweight_gestage.qza \
--o-regression-stats mm_regression-PM25weighted_6m_preprgBMI_BFcont_age_sex_birthweight_gestage.qza \
--o-regression-biplot mm_biplot-PM25weighted_6m_preprgBMI_BFcont_age_sex_birthweight_gestage.qza

# All but gestational age cat
!qiime songbird multinomial \
--i-table mm_seqs_noblank_inf_6m.qza \
--m-metadata-file mm_meta_added_infant_6m.txt \
--p-formula "PM25_weighted+prepreg_bmi_kgm2+breastfeedings_continuous+age_in_days+baby_gender_cat+baby_birthweight_kg" \
--p-epochs 10000 \
--p-differential-prior 0.5 \
--p-summary-interval 1 \
--o-differentials mm_differentials-PM25weighted_6m_preprgBMI_BFcont_age_sex_birthweight.qza \
--o-regression-stats mm_regression-PM25weighted_6m_preprgBMI_BFcont_age_sex_birthweight.qza \
--o-regression-biplot mm_biplot-PM25weighted_6m_preprgBMI_BFcont_age_sex_birthweight.qza

# All but gestational age cat AND age in days
!qiime songbird multinomial \
--i-table mm_seqs_noblank_inf_6m.qza \
--m-metadata-file mm_meta_added_infant_6m.txt \
--p-formula "PM25_weighted+prepreg_bmi_kgm2+breastfeedings_continuous+baby_gender_cat+baby_birthweight_kg" \
--p-epochs 10000 \
--p-differential-prior 0.5 \
--p-summary-interval 1 \
--o-differentials mm_differentials-PM25weighted_6m_preprgBMI_BFcont_sex_birthweight.qza \
--o-regression-stats mm_regression-PM25weighted_6m_preprgBMI_BFcont_sex_birthweight.qza \
--o-regression-biplot mm_biplot-PM25weighted_6m_preprgBMI_BFcont_sex_birthweight.qza

# All but gestational age cat AND sex
!qiime songbird multinomial \
--i-table mm_seqs_noblank_inf_6m.qza \
--m-metadata-file mm_meta_added_infant_6m.txt \
--p-formula "PM25_weighted+prepreg_bmi_kgm2+breastfeedings_continuous+age_in_days+baby_birthweight_kg" \
--p-epochs 10000 \
--p-differential-prior 0.5 \
--p-summary-interval 1 \
--o-differentials mm_differentials-PM25weighted_6m_preprgBMI_BFcont_age_birthweight.qza \
--o-regression-stats mm_regression-PM25weighted_6m_preprgBMI_BFcont_age_birthweight.qza \
--o-regression-biplot mm_biplot-PM25weighted_6m_preprgBMI_BFcont_age_birthweight.qza

# Only BF, Sex, birth weight
!qiime songbird multinomial \
--i-table mm_seqs_noblank_inf_6m.qza \
--m-metadata-file mm_meta_added_infant_6m.txt \
--p-formula "PM25_weighted+baby_birthweight_kg+breastfeedings_continuous+baby_gender_cat" \
--p-epochs 10000 \
--p-differential-prior 0.5 \
--p-summary-interval 1 \
--o-differentials mm_differentials-PM25weighted_6m_BFcont_sex_birthweight.qza \
--o-regression-stats mm_regression-PM25weighted_6m_BFcont_sex_birthweight.qza \
--o-regression-biplot mm_biplot-PM25weighted_6m_BFcont_sex_birthweight.qza

# Only Sex, birth weight
!qiime songbird multinomial \
--i-table mm_seqs_noblank_inf_6m.qza \
--m-metadata-file mm_meta_added_infant_6m.txt \
--p-formula "PM25_weighted+baby_birthweight_kg+baby_gender_cat" \
--p-epochs 10000 \
--p-differential-prior 0.5 \
--p-summary-interval 1 \
--o-differentials mm_differentials-PM25weighted_6m_sex_birthweight.qza \
--o-regression-stats mm_regression-PM25weighted_6m_sex_birthweight.qza \
--o-regression-biplot mm_biplot-PM25weighted_6m_sex_birthweight.qza

# Only Sex, prepreg bmi
!qiime songbird multinomial \
--i-table mm_seqs_noblank_inf_6m.qza \
--m-metadata-file mm_meta_added_infant_6m.txt \
--p-formula "PM25_weighted+prepreg_bmi_kgm2+baby_gender_cat" \
--p-epochs 10000 \
--p-differential-prior 0.5 \
--p-summary-interval 1 \
--o-differentials mm_differentials-PM25weighted_6m_sex_prepregBMI.qza \
--o-regression-stats mm_regression-PM25weighted_6m_sex_prepregBMI.qza \
--o-regression-biplot mm_biplot-PM25weighted_6m_sex_prepregBMI.qza

# PM25 and birthweight
!qiime songbird multinomial \
--i-table mm_seqs_noblank_inf_6m.qza \
--m-metadata-file mm_meta_added_infant_6m.txt \
--p-formula "PM25_weighted+baby_birthweight_kg" \
--p-epochs 10000 \
--p-differential-prior 0.5 \
--p-summary-interval 1 \
--o-differentials mm_differentials-PM25weighted_6m_birthweight.qza \
--o-regression-stats mm_regression-PM25weighted_6mbirthweight.qza \
--o-regression-biplot mm_biplot-PM25weighted_6mbirthweight.qza

# PM25, birthweight, prepregBMI
!qiime songbird multinomial \
--i-table mm_seqs_noblank_inf_6m.qza \
--m-metadata-file mm_meta_added_infant_6m.txt \
--p-formula "PM25_weighted+baby_birthweight_kg+prepreg_bmi_kgm2" \
--p-epochs 10000 \
--p-differential-prior 0.5 \
--p-summary-interval 1 \
--o-differentials mm_differentials-PM25weighted_6m_birthweight_prepregBMI.qza \
--o-regression-stats mm_regression-PM25weighted_6mbirthweight_prepregBMI.qza \
--o-regression-biplot mm_biplot-PM25weighted_6mbirthweight_prepregBMI.qza

# birth weight and prepreg BMI only
!qiime songbird multinomial \
--i-table mm_seqs_noblank_inf_6m.qza \
--m-metadata-file mm_meta_added_infant_6m.txt \
--p-formula "baby_birthweight_kg+prepreg_bmi_kgm2" \
--p-epochs 10000 \
--p-differential-prior 0.5 \
--p-summary-interval 1 \
--o-differentials mm_differentials-NO_PM25weighted_6m_birthweight_prepregBMI.qza \
--o-regression-stats mm_regression-NO_PM25weighted_6m_birthweight_prepregBMI.qza \
--o-regression-biplot mm_biplot-NO_PM25weighted_6m_birthweight_prepregBMI.qza


2020-12-22 13:47:06.669763: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-12-22 13:47:06.679663: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fa60ef773c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-12-22 13:47:06.679695: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version

Instructions for updating:
Use `tf.random.categorical` instead.

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributio

In [120]:
# Visualize the first model's regression stats *and* the null model's
# regression stats
!qiime songbird summarize-paired \
--i-regression-stats mm_regression-PM25weighted_6m_preprgBMI_BFcont_age_sex_birthweight_gestage.qza \
--i-baseline-stats null-stats_6m.qza \
--o-visualization paired-summary-full-vs-null_6m.qzv

# *** Visualize the all but gestational age cat model's regression stats *and* the null model's
# regression stats = Psuedo R2 = 0.08
!qiime songbird summarize-paired \
--i-regression-stats mm_regression-PM25weighted_6m_preprgBMI_BFcont_age_sex_birthweight.qza \
--i-baseline-stats null-stats_6m.qza \
--o-visualization paired-summary-nogestagecat-vs-null_6m.qzv

# Visualize the all but gestational age cat AND age in days model's regression stats *and* the null model's
# regression stats = Psuedo R2 = 0.06
!qiime songbird summarize-paired \
--i-regression-stats mm_regression-PM25weighted_6m_preprgBMI_BFcont_sex_birthweight.qza \
--i-baseline-stats null-stats_6m.qza \
--o-visualization paired-summary-nogestagecat_age-vs-null_6m.qzv

# Visualize the all but gestational age cat AND SEX in days model's regression stats *and* the null model's
# regression stats = Psuedo R2 = 0.06
!qiime songbird summarize-paired \
--i-regression-stats mm_regression-PM25weighted_6m_preprgBMI_BFcont_age_birthweight.qza \
--i-baseline-stats null-stats_6m.qza \
--o-visualization paired-summary-nogestagecat_sex-vs-null_6m.qzv

# Visualize BF, Sex, birth weight only in days model's regression stats *and* the null model's
# regression stats = Psuedo R2 = 0.09 but LOSS is worse
!qiime songbird summarize-paired \
--i-regression-stats mm_regression-PM25weighted_6m_BFcont_birthweight.qza \
--i-baseline-stats null-stats_6m.qza \
--o-visualization paired-summary-onlyBFSexBirthWeight-vs-null_6m.qzv

# Visualize Sex, birth weight only in days model's regression stats *and* the null model's
# regression stats = Psuedo R2 = 0.06 but LOSS is bad
!qiime songbird summarize-paired \
--i-regression-stats mm_regression-PM25weighted_6m_sex_birthweight.qza \
--i-baseline-stats null-stats_6m.qza \
--o-visualization paired-summary-onlySexBirthWeight-vs-null_6m.qzv

# Visualize Sex, birth weight only in days model's regression stats *and* the null model's
# regression stats = Psuedo R2 = 0.07 LOSS in bad
!qiime songbird summarize-paired \
--i-regression-stats mm_regression-PM25weighted_6m_sex_prepregBMI.qza \
--i-baseline-stats null-stats_6m.qza \
--o-visualization paired-summary-onlySexPrePregBMI-vs-null_6m.qzv

# PM25 only
# regression stats
!qiime songbird summarize-paired \
--i-regression-stats mm_regression-PM25weighted_6m.qza \
--i-baseline-stats null-stats_6m.qza \
--o-visualization paired-summary-onlyPM25weighted-vs-null_6m.qzv

# PM25 and birthweight
# regression stats = Psuedo R2 = 0.07 LOSS in bad
!qiime songbird summarize-paired \
--i-regression-stats mm_regression-PM25weighted_6mbirthweight.qza \
--i-baseline-stats null-stats_6m.qza \
--o-visualization paired-summary-onlyPM25weighted_birthweight-vs-null_6m.qzv

# PM25 and birthweight
# regression stats
!qiime songbird summarize-paired \
--i-regression-stats mm_regression-PM25weighted_6mbirthweight_prepregBMI.qza \
--i-baseline-stats null-stats_6m.qza \
--o-visualization paired-summary-onlyPM25weighted_birthweight_prepregBMI-vs-null_6m.qzv

# NO PM25 and only birthweight and prepreg BMI
# regression stats
!qiime songbird summarize-paired \
--i-regression-stats mm_regression-NO_PM25weighted_6m_birthweight_prepregBMI.qza \
--i-baseline-stats null-stats_6m.qza \
--o-visualization paired-summary-NOPM25weighted_birthweight_prepregBMI-vs-null_6m.qzv

Saved Visualization to: paired-summary-full-vs-null_6m.qzv
Saved Visualization to: paired-summary-nogestagecat-vs-null_6m.qzv
Saved Visualization to: paired-summary-nogestagecat_age-vs-null_6m.qzv
Saved Visualization to: paired-summary-nogestagecat_sex-vs-null_6m.qzv
Saved Visualization to: paired-summary-onlyBFSexBirthWeight-vs-null_6m.qzv


Saved Visualization to: paired-summary-NOPM25weighted_birthweight_prepregBMI-vs-null_6m.qzv


In [99]:
# Output results to qzv to visualize cross validation and Loss
#!qiime songbird summarize-single \
#--i-regression-stats mm_regression-PM25weighted_6m.qza \
#--o-visualization mm_regression-PM25weighted_6m.qzv

#!qiime songbird summarize-single \
#--i-regression-stats mm_regression-PM25weighted_6m_preprgBMI_BFcont_age_sex_birthweight_gestage.qza \
#--o-visualization mm_regression-PM25weighted_6m_preprgBMI_BFcont_age_sex_birthweight_gestage.qzv

Saved Visualization to: mm_regression-PM25weighted_6m.qzv
Saved Visualization to: mm_regression-PM25weighted_6m_preprgBMI_BFcont_age_sex_birthweight_gestage.qzv


In [134]:
# Visualize using qurro - need to verify file for --i-table is correct
!qiime qurro differential-plot \
    --i-table mm_seqs_noblank_inf_6m.qza \
    --i-ranks mm_differentials-PM25weighted_6m_preprgBMI_BFcont_age_sex_birthweight_gestage.qza \
    --m-sample-metadata-file mm_meta_added_infant_6m.txt \
    --m-feature-metadata-file deblur-sequences-taxonomy-MM.qza \
    --o-visualization qurro-plot-PM25stats_6m_preprgBMI_BFcont_age_sex_birthweight_gestage.qzv

Saved Visualization to: qurro-plot-PM25stats_6m_preprgBMI_BFcont_age_sex_birthweight_gestage.qzv


In [ ]:
# Visulaizing biplot in emperor
qiime emperor biplot \
--i-biplot mm_test_seqs_regression-PM24biplot_adjPrePregBMI.qza \
--m-sample-metadata-file mm_meta_added_infant.txt \
--p-ignore-missing-samples \
--p-number-of-features 2 \
--o-visualization emperor-biplot-PM25stats_adjPrePregBMI

In [135]:
# Output differentials from Songbird
!qiime tools export \
  --input-path mm_differentials-PM25weighted_6m_preprgBMI_BFcont_age_sex_birthweight_gestage.qza \
  --output-path mm_differentials-PM25weighted_6m_preprgBMI_BFcont_age_sex_birthweight_gestage

Exported mm_differentials-PM25weighted_6m_preprgBMI_BFcont_age_sex_birthweight_gestage.qza as DifferentialDirectoryFormat to directory mm_differentials-PM25weighted_6m_preprgBMI_BFcont_age_sex_birthweight_gestage


In [ ]:
# Can import qza into R directly
# install in R
# devtools::install_github("jbisanz/qiime2R")
# devtools::install_github("joey711/biomformat")
# then load the librairies 
# use read_qza(filename.qza)
# can get alpha diversity measures into metadata
# observed_features_vector = richess (one Justin is adjusting for in analysis) *sig by rapid growers
# Shannon for alpha/eveness
# Faiths PD for phylogentically informed metric *sig by rapid growers
# Dicode index not part of Qiime - Robust Atichson uses matrix completion step to get rid of zeros
# using instead of Attichons or Bray Crutis 
# Bray Curtis has issues dealing with missingness (can make sig when no pattern at all - kind of like random)

In [ ]:
#Cool! Yes I would definitely try un/weighted UniFrac, and also the robust Aitchison distance included in the QIIME2 deicode package (run 'pip install deicode' in your QIIME2 environment). The robust Aitchison distance is the best thing to use when you don't have a phylogeny - so use it instead of Bray-Curtis. Also, I don't expect much clustering in the ordination based on my previous experience with the complete dataset, however I would urge you to run adonis/permanova on the factor representing the three groups (rapid growers, not, and adults) to see if there is any significance. It will also be worth running permdisp on the same factor/dataset. Happy to jump on a call if you need more info here.